## Create condition-specific EV files

In [5]:
import os
from os.path import join
import numpy as np
import pandas as pd
from glob import glob

subs = ['sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-010', 
        'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 
        'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 
        'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 
        'sub-028', 'sub-029', 'sub-030', 'sub-031', 'sub-032']

stim_sets = ['set1', 'set2']

for sub in subs:
    sub_dir = '/home/data/madlab/Mattfeld_vCAT/behav/sub-{0}/'.format(sub)
    dir_files = sorted(glob(join(sub_dir, '{0}_simp_task*.csv'.format(sub)))) 
    print(sub, len(dir_files))
    
    for i, curr_set in enumerate(stim_sets):
        for curr_run in ['1', '2']:
            if curr_run == '1':
                run = pd.read_csv(dir_files[i*2]) #create dataframe for text files to extract EVS
                #run = run[:-3] #removal of the last 3 trials to avoid scanner artifact
            else:
                run = pd.read_csv(dir_files[i*2+1])
                #run = run[:-3]                

            trialtype = run['trialtype'].values #convert dataframes into numpy arrays
            acc = run['acc'].values
            resp = run['resp'].values
            stim = run['stim'].values
            onsets = run['onset'].values
             
            trial_shift = trialtype[1:] #shift trial type back and insert dummy (-1) in last index
            trial_shift = np.append(trial_shift, -1)
            acc_shift = acc[1:] #shift accuracy back and insert dummy (-1) in last index
            acc_shift = np.append(acc_shift, -1)
            resp_shift = resp[1:] #shift response back and insert dummy (-1) in last index
            resp_shift = np.append(resp_shift, -1)
            stim_shift = stim[1:] #shift face/scene back and insert dummy (-1) in last index
            stim_shift = np.append(stim_shift, -1)
            
            #grab indices matching specified criteria
            fixed_cond_corr = np.where(((trial_shift=='cond') & (acc_shift==1)) & (trialtype!='BL'))[0]
            fixed_cond_incorr = np.where((trial_shift=='cond') & (acc_shift==0) & (resp_shift!='None'))[0]
            remaining = list(range(0,180)) #remaining contains trials which dont proceed corr/incorr conds, and nonresp       
            for x in fixed_cond_corr:
                remaining.remove(x)
            for y in fixed_cond_incorr:
                remaining.remove(y)

            #index onsets array using indices from np.where() criteria 
            fixed_cond_corr_onsets = onsets[fixed_cond_corr] 
            fixed_cond_incorr_onsets = onsets[fixed_cond_incorr] 
            remaining_onsets = onsets[remaining]
            
            #vstack matrix containing *ALL* onsets, durations, and amplitudes in vertical columns 
            mtrx = np.vstack((onsets, np.ones(len(onsets))*3.0, #numpy array filled with 3's
                              np.ones(len(onsets)))).T #numpy array filled with 1's
            fixed_cond_corr_mtrx = np.vstack((fixed_cond_corr_onsets, 
                                              np.ones(len(fixed_cond_corr_onsets))*3.0,
                                              np.ones(len(fixed_cond_corr_onsets)))).T 
            fixed_cond_incorr_mtrx = np.vstack((fixed_cond_incorr_onsets,
                                                np.ones(len(fixed_cond_incorr_onsets))*3.0,
                                                np.ones(len(fixed_cond_incorr_onsets)))).T
            remaining_mtrx = np.vstack((remaining_onsets,
                                        np.ones(len(remaining_onsets))*3.0,
                                        np.ones(len(remaining_onsets)))).T

            if not os.path.exists(join(sub_dir, 'GLM1')): #if directory does not exist
                os.makedirs(join(sub_dir, 'GLM1')) #create it
                
            if curr_run == '1': #if first run in stim set
                np.savetxt(join(sub_dir, 'GLM1', 'run{0}.txt'.format(i*2+1)), mtrx, delimiter='\t', fmt='%.4f')                
                for trial in ['fixed_cond_corr', 'fixed_cond_incorr', 'remaining']: #for all trial types
                    exec('np.savetxt(sub_dir+"GLM1/"+"run{0}_{1}.txt",{1}_mtrx,delimiter="\t",fmt="%.4f")'.format(i*2+1,trial))

            else: #if second run in stim set
                np.savetxt(join(sub_dir, 'GLM1', 'run{0}.txt'.format(i*2+2)), mtrx, delimiter='\t', fmt='%.4f')                
                for trial in ['fixed_cond_corr', 'fixed_cond_incorr', 'remaining']:
                    exec('np.savetxt(sub_dir+"GLM1/"+"run{0}_{1}.txt",{1}_mtrx,delimiter="\t",fmt="%.4f")'.format(i*2+2,trial))

/home/data/madlab/envs/wmaze_madlab_env/lib/python2.7/site-packages/ipykernel/__main__.py:43: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/home/data/madlab/envs/wmaze_madlab_env/lib/python2.7/site-packages/ipykernel/__main__.py:42: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
